In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

import argparse
import csv
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import loguniform
from scipy.io import loadmat
from sklearn import metrics
from torch.utils import data
from IPython.display import clear_output

#sys.path.append('/content/drive/MyDrive/Tesi_Magistrale')
sys.path.append('D:/Tesi_Magistrale')
from Data_Processing_Utils import Norm_each_sub_by_own_param,NormalizeData, train_val_test_split_df, PlotLoss, plot_confusion_matrix, \
    Get_Sub_Norm_params, windowing_Dataframe

from DATALOADERS import dataframe_dataset_triplet, Pandas_Dataset

from MODELS import MKCNN, MKCNN_grid, random_choice, train_model_triplet, pre_train_model_triplet, train_model_standard, MultiKernelConv2D_grid
pd.options.mode.chained_assignment = None

In [2]:
# custom pandas dataset class modified to provide also the angle
class Pandas_Dataset_angle(data.Dataset):

    def __init__(self, df_grouped_by_samples, return_sub=False):
        self.grouped = df_grouped_by_samples
        self.channels = [i for i in df_grouped_by_samples.obj.columns if 'ch' in i]
        self.indices = list(df_grouped_by_samples.indices)
        self.return_sub = return_sub

    def __len__(self):
        return len(self.grouped)

    def __getitem__(self, index):
        picked_smp = self.grouped.get_group(self.indices[index])
        # Picking only the channels columns from a single sample
        sample = torch.tensor(picked_smp.loc[:, self.channels].values).type(torch.float32)
        # picking only one label for each sample
        label = torch.tensor(picked_smp.loc[:, ['label']].head(1).values[0][0]).type(torch.int8)
        angle = torch.tensor(picked_smp.loc[:, ['angle']].head(1).values[0][0]).type(torch.int16)
        if self.return_sub:
            # picking only one subject
            sub = torch.tensor(picked_smp.loc[:, ['sub']].head(1).values[0][0]).type(torch.int8)
            return sample, label, angle, sub
        else:

            # It's missing the part in which I Normalize the data for each subject, or for each subject and channels
            # It's not a good idea to do that on the dataloader while producing results
            return sample, label, angle


In [6]:
#file_path = '/content/drive/MyDrive/Tesi_Magistrale'
#data_path = '/content/drive/MyDrive/Tesi_Magistrale/DATA/'

file_path = 'D:/Tesi_Magistrale'
data_path = 'D:/Tesi_Magistrale/DATA/'

subjects = ['Sub1','Sub2','Sub3']
#subj = 'Manfredo'
day = 1
single_model = False

# constants
WND_LEN = 200
NORM_MODE = 'sub'
NORM_TYPE = 'zero_to_one'
RECT = True
split_mode = 'rep'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

exe_labels = ['medium_wrap', 'lateral', 'power_sphere', 'power_disk', 'prismatic_pinch', 'index_extension',
                    'wave_out', 'wave_in', 'fist', 'open_hand']
'''
test1
[[[1,2,3,4],[5],[6]],
 [[1,2,3],[4],[5,6]],
 [[1,2],[3],[4,5,6]],
 [[1],[2],[3,4,5,6]]]
test2
[[[1,2,5,6],[3],[4]],
 [[1,2,5],[3],[4,6]],
 [[2,5],[3],[1,4,6]],
 [[2],[3],[1,4,5,6]]]
test3
[[[2,4,6],[1,5],[3]],
 [[2,6],[1,5],[3,4]],
 [[2],[1,5],[3,4,6]]]

'''

test = 2
train_val_test_rep_combo = [[[1,2,5,6],[3],[4]],
 [[1,2,5],[3],[4,6]],
 [[2,5],[3],[1,4,6]],
 [[2],[3],[1,4,5,6]]]



Using device: cuda


In [7]:
for trial in range (3,4):
    for subj in subjects:
        df = pd.read_csv(data_path+f'{subj}_offset_relabel/day{day}/Dataframe/dataframe_wnd_{WND_LEN}_angle_merged.csv')
        df = Norm_each_sub_by_own_param(df, mode= NORM_MODE, norm_type= NORM_TYPE, rectify=RECT )
        for train_val_test_rep in train_val_test_rep_combo:
            test_reps = train_val_test_rep[2]
            save_dir_name = ''
            for l in train_val_test_rep:
                for x in l:
                  save_dir_name = save_dir_name + f'{x}'
                save_dir_name = save_dir_name+'_'
            save_dir_name = save_dir_name[0:-1]
            save_path = data_path + f'/{subj}_offset_relabel/day{day}/Prova_{trial}/Rep_Tests/test{test}/{save_dir_name}/'
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            print(f"Using Train:{train_val_test_rep[0]}, Val:{train_val_test_rep[1]}, Test:{train_val_test_rep[2]}\nSingle Model: {single_model}")
            print(f'Saving in folder "{save_path}"\n')
            print(f"Starting Trial {trial} for sub {subj}")
            
            # single model
            print("Training single model")
            n_channels = [cx for cx in df.columns if 'ch' in cx]
            df.loc[:, n_channels] = df.loc[:, n_channels].astype(np.float32)
    
            df_train, df_val, df_test = train_val_test_split_df(df, mode=split_mode, manual_sel=train_val_test_rep)
            train_set = Pandas_Dataset(df_train.groupby('sample_index'))
            valid_set = Pandas_Dataset(df_val.groupby('sample_index'))
            test_set = Pandas_Dataset(df_test.groupby('sample_index'))
    
            valid_group = np.unique(df_val[split_mode])
            test_group = np.unique(df_test[split_mode])
    
            # Dataloaders
            batch_size = 32
            num_workers = 0
            params = {'batch_size': batch_size,
                    'shuffle': True,
                    # 'sampler': sampler,
                    'num_workers': num_workers,
                    'drop_last': False}
    
            train_set_generator = data.DataLoader(train_set, **params)
            valid_set_generator = data.DataLoader(valid_set, **params)
            test_set_generator = data.DataLoader(test_set, **params)
    
            # Model
            kernels_gap = [g for g in range(0, 3 * round(WND_LEN / 20), round(WND_LEN / 20))]
            kernel_sizes = np.full((3, 5, 2), [1, 3])
            for j in range(3):
                for i in range(5):
                    kernel_sizes[j][i][0] = (round(WND_LEN / 20) * (i + 1) + kernels_gap[j])
    
            n_classes = 10
            net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
            'Kernel_multi_dim': kernel_sizes[0],
            'Kernel_Conv_conc': 1,
            'act_func': nn.ReLU(),
            'Pool_Type': nn.MaxPool2d,
            'wnd_len':WND_LEN
            }
    
            model = MKCNN_grid(net, num_classes=n_classes)
            model = model.to(device)
    
            #%% Loss Optim and scheduler
            # Define Loss functions
            cross_entropy_loss = nn.CrossEntropyLoss(reduction='mean').to(device)
            # triplet = nn.TripletMarginLoss(reduction='mean', margin=1, p=2)
    
            # Define Optimizer
            learning_rate = 0.0001
            # changed beta values from (0.5,0.999) to (0.9,0.999)
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-4)
    
            # # Define Scheduler
            precision = 1e-6
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=.2,
                                                                patience=5, verbose=True, eps=precision)
    
            print(f'Model created with Num params: {sum([p.numel() for p in model.parameters() if p.requires_grad])}')
    
            # Start Training
            best_weights, tr_losses, val_losses = train_model_standard(model=model, loss_fun=cross_entropy_loss,
                                                                optimizer=optimizer, scheduler=scheduler,
                                                                dataloaders={"train": train_set_generator,
                                                                                "val": valid_set_generator},
                                                                num_epochs=50, precision=precision,
                                                                patience=10, patience_increase=5)
    
            #%% Saving
            database = save_path
            filename = f'wnd_{WND_LEN}_{NORM_TYPE}_{NORM_MODE}_rect_{RECT}_single_model_day{day}'
            # Save state dict of the model
            if not os.path.exists(database + f'Cross_{split_mode}/Best_States/'):
                os.makedirs(database + f'Cross_{split_mode}/Best_States/')
            torch.save(best_weights['state_dict'],
                    database + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth')
    
            # %% PlotLoss
            if not os.path.exists(database + f'Cross_{split_mode}/Plot/'):
                os.makedirs(database + f'Cross_{split_mode}/Plot/')
    
            PlotLoss(tr_losses, val_loss=val_losses,
                    title=f'Cross-{split_mode}',
                    path_to_save=database + f'Cross_{split_mode}/Plot/',
                    filename=f'Cross_{split_mode}_{filename}.png')
    
            # %% Conf Matrix
            if not os.path.exists(database + f'Cross_{split_mode}/Conf_Matrix/'):
                os.makedirs(database + f'Cross_{split_mode}/Conf_Matrix/')
            clear_output()
            # Evaluation
            softmax_block = nn.Softmax(dim=1)
            y_true = []
            y_pred = []
    
            test_dataloader = test_set_generator
            net = model
            # Load Weights
            net.load_state_dict(torch.load(database + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth'))
    
            net.eval()
            with torch.no_grad():
                for inputs, labels in test_dataloader:
                    # inputs = torch.swapaxes(inputs, 2, 1)  # -> convert from [10,20] to [20,10]
                    inputs = inputs[:, None, :, :]
                    inputs = inputs.to(device)
                    labels_np = labels.cpu().data.numpy()
                    # forward
                    outputs, _ = net(inputs)
                    outputs_np = softmax_block(outputs)
                    outputs_np = outputs_np.cpu().data.numpy()
                    outputs_np = np.argmax(outputs_np, axis=1)
    
                    y_pred = np.append(y_pred, outputs_np)
                    y_true = np.append(y_true, labels_np)
    
                cm = metrics.confusion_matrix(y_true=y_true, y_pred=y_pred)
                # Fancy confusion matrix
                plot_confusion_matrix(cm, target_names=exe_labels, title=f'Confusion Matrix for {split_mode} {test_group}',
                                    path_to_save=database + f'Cross_{split_mode}/Conf_Matrix/Cross_{split_mode}_N_{filename}.png')
    
            # %% Write in csv cross_sub results
            # Building columns
            header_net = [f'Tested_{split_mode}', 'Best Val Loss', 'Accuracy', 'Kappa', 'F1_score', 'Best Epoch', 'Norm_Type',
                        'Norm_Mode', 'Rect', f'Valid_{split_mode}']
    
            # Open the CSV file and write the headers and row of values
            with open(database + f'Cross_{split_mode}/Evals_single_model.csv', 'a', newline='') as myFile:
                writer = csv.writer(myFile)
                if myFile.tell() == 0:
                    writer.writerow(header_net)
                # Create the row of values
                row = [test_group, min(val_losses),
                    metrics.accuracy_score(y_true=y_true, y_pred=y_pred),
                    metrics.cohen_kappa_score(y1=y_true, y2=y_pred, weights='quadratic'),
                    metrics.f1_score(y_true=y_true, y_pred=y_pred, average='macro'),
                    best_weights['epoch'], NORM_TYPE, NORM_MODE, str(RECT), valid_group]
                writer.writerow(row)
                print(f'Results Saved in -> {database}/Cross_{split_mode}/Evals.csv')
            clear_output()
            
            
            
            # multi angle model
            for ANGLE in range(1,6):
                print(f'Training angle {ANGLE}')
                df_train, df_val, df_test = train_val_test_split_df( df[df['angle_index'] == ANGLE] , mode=split_mode, manual_sel=train_val_test_rep)
    
                # Saving val and test rep used
                valid_group = np.unique(df_val[split_mode])
                test_group = np.unique(df_test[split_mode])[0]
                # %% pandas dataset
                # You must pass to the dataloader a groupby dataframe
                train_set = Pandas_Dataset(df_train.groupby('sample_index'))
                valid_set = Pandas_Dataset(df_val.groupby('sample_index'))
                test_set = Pandas_Dataset(df_test.groupby('sample_index'))
    
                # %% Dataloaders
                batch_size = 32
                num_workers = 0
                params = {'batch_size': batch_size,
                        'shuffle': True,
                        # 'sampler': sampler,
                        'num_workers': num_workers,
                        'drop_last': False}
    
                train_set_generator = data.DataLoader(train_set, **params)
                valid_set_generator = data.DataLoader(valid_set, **params)
                test_set_generator = data.DataLoader(test_set, **params)
    
                #%% Parametric model
                kernels_gap = [g for g in range(0, 3 * round(WND_LEN / 20), round(WND_LEN / 20))]
                kernel_sizes = np.full((3, 5, 2), [1, 3])
                for j in range(3):
                    for i in range(5):
                        kernel_sizes[j][i][0] = (round(WND_LEN / 20) * (i + 1) + kernels_gap[j])
    
                n_classes = 10
    
                net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
                'Kernel_multi_dim': kernel_sizes[0],
                'Kernel_Conv_conc': 1,
                'act_func': nn.ReLU(),
                'Pool_Type': nn.MaxPool2d,
                'wnd_len':WND_LEN
                }
    
                model = MKCNN_grid(net, num_classes=n_classes)
                model = model.to(device)
    
                #%% Loss Optim and scheduler
                # Define Loss functions
                cross_entropy_loss = nn.CrossEntropyLoss(reduction='mean').to(device)
                # triplet = nn.TripletMarginLoss(reduction='mean', margin=1, p=2)
        
                # Define Optimizer
                learning_rate = 0.0001
                # changed beta values from (0.5,0.999) to (0.9,0.999)
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=1e-4)
        
                # # Define Scheduler
                precision = 1e-6
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=.2,
                                                                    patience=5, verbose=True, eps=precision)
    
                print(f'Model created with Num params: {sum([p.numel() for p in model.parameters() if p.requires_grad])}')
    
                # Training
                best_weights, tr_losses, val_losses = train_model_standard(model=model, loss_fun=cross_entropy_loss,
                                                                      optimizer=optimizer, scheduler=scheduler,
                                                                      dataloaders={"train": train_set_generator,
                                                                                      "val": valid_set_generator},
                                                                      num_epochs=50, precision=precision,
                                                                      patience=10, patience_increase=5)
                #%% Saving
                database_angle = save_path
                filename = f'wnd_{WND_LEN}_{NORM_TYPE}_{NORM_MODE}_rect_{RECT}_ang_{ANGLE}_day{day}'
    
                # Save state dict of the model
                if not os.path.exists(database_angle + f'Cross_{split_mode}/Best_States/'):
                    os.makedirs(database_angle + f'Cross_{split_mode}/Best_States/')
                torch.save(best_weights['state_dict'],
                        database_angle + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth')
    
                # %% PlotLoss
                if not os.path.exists(database_angle + f'Cross_{split_mode}/Plot/'):
                    os.makedirs(database_angle + f'Cross_{split_mode}/Plot/')
    
                PlotLoss(tr_losses, val_loss=val_losses,
                        title=f'Cross-{split_mode}',
                        path_to_save=database_angle + f'Cross_{split_mode}/Plot/',
                        filename=f'Cross_{split_mode}_{filename}.png')
    
                # %% Conf Matrix
                if not os.path.exists(database_angle + f'Cross_{split_mode}/Conf_Matrix/'):
                    os.makedirs(database_angle + f'Cross_{split_mode}/Conf_Matrix/')
    
                # Evaluation
                softmax_block = nn.Softmax(dim=1)
                y_true = []
                y_pred = []
    
                test_dataloader = test_set_generator
                net = model
                # Load Weights
                net.load_state_dict(torch.load(database_angle + f'Cross_{split_mode}/Best_States/state_dict_{filename}.pth'))
    
                net.eval()
                with torch.no_grad():
                    for inputs, labels in test_dataloader:
                        # inputs = torch.swapaxes(inputs, 2, 1)  # -> convert from [10,20] to [20,10]
                        inputs = inputs[:, None, :, :]
                        inputs = inputs.to(device)
                        labels_np = labels.cpu().data.numpy()
                        # forward
                        outputs, _ = net(inputs)
                        outputs_np = softmax_block(outputs)
                        outputs_np = outputs_np.cpu().data.numpy()
                        outputs_np = np.argmax(outputs_np, axis=1)
    
                        y_pred = np.append(y_pred, outputs_np)
                        y_true = np.append(y_true, labels_np)
    
                    cm = metrics.confusion_matrix(y_true=y_true, y_pred=y_pred)
                    # Fancy confusion matrix
                    plot_confusion_matrix(cm, target_names=exe_labels, title=f'Confusion Matrix for {split_mode} {test_group}',
                                        path_to_save=database_angle + f'Cross_{split_mode}/Conf_Matrix/Cross_{split_mode}_N_{filename}.png')
    
                # %% Write in csv cross_sub results
                # Building columns
                header_net = [f'Tested_{split_mode}', 'Best Val Loss', 'Accuracy', 'Kappa', 'F1_score', 'Best Epoch', 'Norm_Type',
                            'Norm_Mode', 'Rect', f'Valid_{split_mode}']
    
                # Open the CSV file and write the headers and row of values
                with open(database_angle + f'Cross_{split_mode}/Evals_multi_model_angle_{ANGLE}.csv', 'a', newline='') as myFile:
                    writer = csv.writer(myFile)
                    if myFile.tell() == 0:
                        writer.writerow(header_net)
                    # Create the row of values
                    row = [test_group, min(val_losses),
                        metrics.accuracy_score(y_true=y_true, y_pred=y_pred),
                        metrics.cohen_kappa_score(y1=y_true, y2=y_pred, weights='quadratic'),
                        metrics.f1_score(y_true=y_true, y_pred=y_pred, average='macro'),
                        best_weights['epoch'], NORM_TYPE, NORM_MODE, str(RECT), valid_group]
                    writer.writerow(row)
                    print(f'Results Saved in -> {database_angle}/Cross_{split_mode}/Evals.csv')
                    myFile.close()
                clear_output()
            net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
            'Kernel_multi_dim': kernel_sizes[0],
            'Kernel_Conv_conc': 1,
            'act_func': nn.ReLU(),
            'Pool_Type': nn.MaxPool2d,
            'wnd_len':WND_LEN
            }
            print("Starting Multimodel Testing")
            m1 = MKCNN_grid(net, num_classes=n_classes)
            m2 = MKCNN_grid(net, num_classes=n_classes)
            m3 = MKCNN_grid(net, num_classes=n_classes)
            m4 = MKCNN_grid(net, num_classes=n_classes)
            m5 = MKCNN_grid(net, num_classes=n_classes)
            m1 = m1.to(device)
            m2 = m2.to(device)
            m3 = m3.to(device)
            m4 = m4.to(device)
            m5 = m5.to(device)
            print("Models Created")
            ws = []
            for ANGLE in range(1,6):
                ws.append(torch.load(save_path + f'Cross_{split_mode}/Best_States/state_dict_wnd_{WND_LEN}_zero_to_one_sub_rect_True_ang_{ANGLE}_day{day}.pth'))
            m1.load_state_dict(ws[0])
            m2.load_state_dict(ws[1])
            m3.load_state_dict(ws[2])
            m4.load_state_dict(ws[3])
            m5.load_state_dict(ws[4])
            print("Weights Loaded")
            
            # Prepare test_df
            test_df = []
            for test_rep in test_reps:
                test_df.append(df[df['rep'] == test_rep])
            if len(test_df)>1:
                test_df = pd.concat(test_df)
            else:
                test_df = test_df[0]

            
            minAngle, maxAngle = test_df['angle'].abs().min(), test_df['angle'].abs().max()
            print(f"minAngle: {minAngle} maxAngle: {maxAngle}")
            angleRange = maxAngle - minAngle
            interval = 5*round(angleRange * 0.125/5) #round interval to the closest multiple of 5
            residue = angleRange-interval*8
            intervals = [abs(minAngle+interval), abs(minAngle+interval*3), abs(maxAngle-interval*3), abs(maxAngle-interval)]
            print(f"Angle Intervals: {intervals}")
            
            test_df = Pandas_Dataset_angle(test_df.groupby('sample_index'))
            batch_size = 1
            num_workers = 0
            params = {'batch_size': batch_size,
                        'shuffle': False,
                        # 'sampler': sampler,
                        'num_workers': num_workers,
                        'drop_last': False}
            test_df_dataloader = data.DataLoader(test_df, **params)
            
            m1.eval()
            m2.eval()
            m3.eval()
            m4.eval()
            m5.eval()
            with torch.no_grad():
                y_pred = []
                y_true = []
                for inputs, labels, angle in test_df_dataloader:
                    inputs = inputs[:, None, :, :]
                    inputs = inputs.to(device)
                    angle = abs(angle.cpu().data.numpy()[0])
                    labels_np = labels.cpu().data.numpy()
                    if angle<intervals[0]:
                        output, _ = m1.forward(inputs)
                        model = 1
                    elif angle<intervals[1]:
                        output, _ = m2.forward(inputs)
                        model = 2
                    elif angle<intervals[2]:
                        output, _ = m3.forward(inputs)
                        model = 3
                    elif angle<intervals[3]:
                        output, _ = m4.forward(inputs)
                        model = 4
                    else:
                        output, _ = m5.forward(inputs)
                        model = 5
                    outputs_np = softmax_block(output)
                    outputs_np = outputs_np.cpu().data.numpy()
                    outputs_np = np.argmax(outputs_np, axis=1)
                    y_pred = np.append(y_pred, outputs_np)
                    y_true = np.append(y_true, labels_np)
                    print(f"Model: {model}\tAngle: {angle}\tPredicted pose: {outputs_np}\tReal Pose: {labels_np}", end = '\r')
            
                print("\nTesting Done!")
            print("Saving Confusion Matrix")
            from Data_Processing_Utils import  plot_confusion_matrix
            
            poses = ['medium_wrap', 'lateral','power_sphere', 'power_disk', 'prismatic_pinch',
                        'index_extension', 'wave_out', 'wave_in', 'fist', 'open_hand']
            
            cm = metrics.confusion_matrix(y_true, y_pred)
            title = f"Confusion Matrix Multi Model {test_rep}" # title on the confusion matrix
            plot_confusion_matrix(cm, target_names=poses, title=f'CM {title} {NORM_TYPE}',
                                            path_to_save=save_path + f'Cross_rep/Conf_Matrix/Cross_rep_N_wnd_{WND_LEN}_{NORM_TYPE}_{NORM_MODE}_rect_{RECT}_multi_model_day{day}.png')
        



Starting Multimodel Testing
Models Created
Weights Loaded
minAngle: 0 maxAngle: 115
Angle Intervals: [15, 45, 70, 100]
Model: 5	Angle: 115	Predicted pose: [9]	Real Pose: [9]
Testing Done!
Saving Confusion Matrix
Conf_Matrix saved in -->  D:/Tesi_Magistrale/DATA//Manfredo_offset_relabel/day1/Prova_3/Rep_Tests/test2/2_3_1456/Cross_rep/Conf_Matrix/Cross_rep_N_wnd_200_zero_to_one_sub_rect_True_multi_model_day1.png
